# **CQoT Pipeline**

# **CQoT Pipeline as a Class**

In [ ]:
import torch, accelerate, flash_attn
import os, re
import subprocess
#remember to upgrade the transformers library to the latest update
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig


class CQoT_Pipeline:
    #when instantiating the class, you must specify a model, but you can also customise attention implementation and torch_dtype
    def __init__(self, model_id, attn_implementation="eager",torch_dtype="auto"):
        self.llm_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", attn_implementation=attn_implementation, torch_dtype=torch_dtype, trust_remote_code=True)
        self.llm_tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.pipe = pipeline("text-generation", model=self.llm_model, tokenizer=self.llm_tokenizer)

    ###########  CLASS VARIABLES DEFINITIONS ###########
    #class variable for reasoning plan generation
    generation_config = {
        "return_full_text": False, #only return the generated text without prepending input
        "temperature": 0.8, #determines the 'creativity' of the model, lower value = more deterministic 
        "top_p": 0.95, #enables nucleus sampling. Keeps cumulative probability of top choices ≤ 0.95
        "do_sample": True, #ensures sampling is used instead of greedy decoding
        "max_new_tokens":2000 #limits the number of new tokens generated in the output
    }

    #class variable for CQs probing and final output generation
    generation_config_zero_temperature = {
        "return_full_text": False, #only return the generated text without prepending input
        "temperature": 0.2, #determines the 'creativity' of the model, lower value = more deterministic 
        "top_p": 0.95, #enables nucleus sampling. Keeps cumulative probability of top choices ≤ 0.95
        "do_sample": True, #ensures sampling is used instead of greedy decoding
        "max_new_tokens":2000 #limits the number of new tokens generated in the output
    }

    
    
    
    ###########  CQoT:Plan Reasoning Steps ###########
    #function to plan reasoning steps
    def get_plan(self, user_prompt):
        system_instruction= """You are a critical and analytical reasoner. Carefully read the user prompt. 
        Instead of replying, provide your step by step reasoning, clearly dividing and outlining the different steps. 
        Each step must be very specific, easy to follow and the set of premises should logically lead to a truthful conclusion. 
        Do NOT provide the final answer (but don't write down 'do not provide the final answer' in the reasoning process)."""

        #specify prompt
        messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": user_prompt+ "Remember not to provide the final answer, just spell out the reasoning plan."},
        ]
        
        #prompt the model to generate content
        response = self.pipe(messages, **self.generation_config)
        #return model response
        return response[0]['generated_text']

    
    
    
    ###########  CQoT:Assess Reasoning Steps ###########
    #function to assess reasoning steps
    def assess_plan(self, reasoning_steps, user_prompt):
        system_instruction= """You are an analytical and critical reasoner. Assess the provided reasoning steps against the following questions using the user prompt as a context. 
        Reply simply with 'Yes' or 'No' for each question. Be very strict and strongly critical and answer 'No' when in doubts. Do NOT answer or reply to anything else than this.
        1.Does the reasoning process start with clearly defined premises?
        2.Are the premises supported by evidence or accepted facts?
        3.Does the reasoning process use logical connections between premises and conclusions?
        4.Are the logical connections used in the reasoning process valid?
        5.Does the reasoning process avoid fallacies or logical errors?
        6.Is the conclusion logically derived from the premises?
        7.Is the reasoning process consistent with established knowledge or principles?
        8.Does the reasoning process lead to a conclusion that is plausible and reasonable?"""

        #specify input
        messages = [
        {"role": "system", "content": system_instruction},
        {"role": "assistant", "content": reasoning_steps},
        {"role": "user", "content": user_prompt+ "Remember: you MUST reply to EACH question. The replies should be ONLY 'Yes' or 'No'"},
        ]
        
        #prompt the model to generate content
        response = self.pipe(messages, **self.generation_config_zero_temperature)
        #return model response
        return response[0]['generated_text']




    ###########  CQoT:Actual Output via Reasoning Plan ###########
    #function to assess reasoning steps
    def actual_output(self, reasoning_steps, user_prompt):
        system_instruction= """You are a skilled, critical and analytical reasoner. You always reason in steps according to the provided instruction. 
        Give an answer to the user prompt by utterly and strictly following the protocol established by the reasoning steps. Here are the stages of what you have to do:
        1) Carefully read and understand the protocol detailed by the reasoning steps. Then, start your reasoning.
        2) Each intermediate conclusion of your reasoning MUST be yielded by the truthfulness of its respective premises.
        3) Each established conclusion MUST NOT contradict other information. After each step, look back at previous steps and check that you are not contradicting what you previously deduced.
        4) Your final reply MUST be logically entailed by the reasoning steps and each established conclusion. Meticulously ensure that your reply is consistent with your reasoning."""

        #specify input
        messages = [
        {"role": "system", "content": system_instruction},
        {"role": "assistant", "content": reasoning_steps},
        {"role": "user", "content": user_prompt},
        ]
        #prompt the model to generate content
        response = self.pipe(messages, **self.generation_config_zero_temperature)
        #return model response
        return response[0]['generated_text']

    

    #########  MAIN METHOD THAT ACTIVATES CQoT_Pipeline ###########
    #set verbose=False to prevent printing intermediate steps
    def activate(self, user_prompt, verbose=True):
      reasoning_plan = self.get_plan(user_prompt)
      if verbose==True:
        print("#############################################")
        print(f'PLAN: {reasoning_plan}')
      verified_plan = self.assess_plan(reasoning_plan, user_prompt)
      if verbose==True:
        print("#############################################")
        print(f'VERIFICATION: {verified_plan}')
      assessment = re.findall("Yes", verified_plan)
      while_counter = 0
      while len(assessment)<7:
          if while_counter<4:
              reasoning_plan = self.get_plan(user_prompt)
              verified_plan = self.assess_plan(reasoning_plan, user_prompt)
              assessment = re.findall("Yes", verified_plan)
              if verbose==True:
                print(f'PLAN: {reasoning_plan}')
                print(f'VERIFICATION: {verified_plan}' + f'\n counter: {str(while_counter)}')
              while_counter += 1
          else:
              reasoning_plan = self.get_plan(user_prompt)
              verified_plan = self.assess_plan(reasoning_plan, user_prompt)
              assessment = re.findall("Yes", verified_plan)
              if verbose==True:
                print(f'PLAN: {reasoning_plan}')
                print(f'VERIFICATION: {verified_plan}' + f'\n counter: {str(while_counter)}')
              while_counter += 1
              if len(assessment)>4:
                  break
              elif while_counter==10:
                  break
              
      final_plan = reasoning_plan
      output = self.actual_output(final_plan, user_prompt) 
      print("#############################################")
      print(f'OUTPUT: {output}')
      return output


    
    #########  METHOD TO GET REASONING_STEP ONLY WITHOUT CQs PROBING ###########
    def reasoning_step_only(self, user_prompt):
        reasoning_plan = self.get_plan(user_prompt)
        output = self.actual_output(reasoning_plan, user_prompt)
        print("#############################################")
        print(f'OUTPUT: {output}')
        return output
             

In [ ]:
#EXAMPLE: CQoT = CQoT_Pipeline("meta-llama/Llama-3.1-70B-Instruct", attn_implementation='flash_attention_2',torch_dtype=torch.float16)
CQoT = CQoT_Pipeline("") #specify at least the model

In [ ]:
pipeline_output = CQoT.activate(""" "Each problem consists of three statements. Based on the first two statements, the third statement may be true, false, or uncertain.\n1. Oranges cost more than apples.\n2. Oranges cost less than bananas.\n3. Bananas cost more than apples and bananas cost more than orange.\nIf the first two statements are true, then the third statement is" """)